# Deep Learning Lotto implementation in Keras

<a href="http://www.crest.fr/ckfinder/userfiles/files/Pageperso/vcottet/paper%20-%20Deep%20Learning%20predicts%20Loto.pdf">Paper</a>
<br>
<a href="
https://stackoverflow.com/questions/44202627/keras-model-from-nn-schematic">StackOverflow</a>

## Setup

In [2]:
# To support both python 2 and python 3
from __future__ import division, print_function, unicode_literals

# Common imports
import numpy as np
import os
import pandas as pd

# to make this notebook's output stable across runs
np.random.seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# Where to save the figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "end_to_end_project"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

# Ignore useless warnings (see SciPy issue #5998)
import warnings
warnings.filterwarnings(action="ignore", message="^internal gelsd")


## Logging

import logging
import io
import contextlib
import sys

@contextlib.contextmanager
def capture_logs():
  """Save logs to an io.StringIO."""
  ip = get_ipython()
  s = io.StringIO()
  h = logging.StreamHandler(s)
  ip.log.addHandler(h)
  try:
    yield s
  finally:
    ip.log.removeHandler(h)

@contextlib.contextmanager
def print_logs():
  """Print logs inline."""  
  ip = get_ipython()
  h = logging.StreamHandler(sys.stdout)
  ip.log.addHandler(h)
  try:
    yield
  finally:
    ip.log.removeHandler(h)
    

ip = get_ipython()

## Set logging level
ip.log.setLevel(logging.INFO)


with capture_logs() as s:
  ip.log.warning('capture logs')
print(s.getvalue())

with print_logs():
  ip.log.info('print logs')

capture logs

print logs


## Get the data

In [0]:
import os
import zipfile
from six.moves import urllib

DOWNLOAD_ROOT = "http://www.sportstoto.com.my/"
LOTTO_PATH = os.path.join(os.getcwd(), "datasets", "lotto")
LOTTO_URL = DOWNLOAD_ROOT + "upload/Toto655.zip"
LOTTO_ZIP = os.path.join(LOTTO_PATH, "lotto.zip")

# print(LOTTO_URL)

# def fix_bad_zip_file(zip_file):  
#  f = open(zip_file, 'r+b')  
#  data = f.read()  
#  pos = data.find('\x50\x4b\x05\x06') # End of central directory signature  
#  if (pos > 0):  
#      self._log("Trancating file at location " + str(pos + 22)+ ".")  
#      f.seek(pos + 22)   # size of 'ZIP end of central directory record' 
#      f.truncate()  
#      f.close()  
#  else:  
#      # raise error, file is truncated  
#     raise RuntimeError('bad zip file')

def fetch_data(url=LOTTO_URL, path=LOTTO_PATH, zip_path=LOTTO_ZIP):
  if not os.path.isdir(path):
      os.makedirs(path)
  urllib.request.urlretrieve(url, zip_path)
  if (os.path.isfile(zip_path)):
    print(f'Download path: {zip_path}')
  # fix_bad_zip_file(zip_path)
  # with zipfile.ZipFile(zip_path, 'r') as lotto_zip:
  #  lotto_zip.extractall(path)

In [4]:
!rm /content/datasets/lotto/lotto.zip

fetch_data()

rm: cannot remove '/content/datasets/lotto/lotto.zip': No such file or directory
Download path: /content/datasets/lotto/lotto.zip


In [0]:
df = pd.read_csv(LOTTO_ZIP, compression='infer', header=0, sep=',', quotechar='"', dtype=str)

In [8]:
df.head(10)

,DrawNo,DrawDate,DrawnNo1,DrawnNo2,DrawnNo3,DrawnNo4,DrawnNo5,DrawnNo6,Jackpot
0,325209,20091031,17,20,24,30,49,50,0426643629
1,325309,20091101,01,05,18,23,50,51,0439869056
2,325409,20091103,05,09,13,15,48,55,0454276056
3,325509,20091104,09,28,36,41,47,50,0466824556
4,325609,20091107,17,19,24,27,33,38,0480533325
5,325709,20091108,17,22,30,32,38,54,0494376831
6,325809,20091111,02,18,25,32,37,54,0508774143
7,325909,20091114,06,13,21,38,41,45,0523964676
8,326009,20091115,01,02,32,36,45,51,0538291775
9,326109,20091117,06,10,31,40,44,54,0550894329
